# Sequence example

>https://github.com/dennybritz/tf-rnn/blob/master/sequence_example.ipynb

In [1]:
import tensorflow as tf
import numpy as np
import tempfile

/Users/Varela/anaconda/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
sequences= [[1,2,3], [4,5,1], [1,2]]
label_sequences= [[0,1,0], [1,0,0], [1,1]]

def make_example(sequences, labels):
    # The object we return
    ex= tf.train.SequenceExample() 
    # A non-sequential feature of our example
    sequence_length=len(sequences)
    ex.context.feature['length'].int64_list.value.append(sequence_length)
    #Feature lists for the two sequential feature of our example
    f1_tokens= ex.feature_lists.feature_list['tokens']
    f1_labels= ex.feature_lists.feature_list['labels']
    for token, label in zip(sequence, labels):
        f1_tokens.feature.add().int64_list.value.append(token)
        f1_labels.feature.add().int64_list.value.append(label)
    return ex

#Write all examples into a TFRecords file
with tempfile.NamedTemporaryFile() as fp:
    writer= tf.python_io.TFRecordWriter(fp.name)
    for sequence, label_sequence in zip(sequences, label_sequences):
        ex= make_example(sequence, label_sequence)
        writer.write(ex.SerializeToString())
    writer.close()
    print('Wrote to {}'.format(fp.name))


Wrote to /var/folders/r6/0pcqbs655t19b_392h5m3x4c0000gn/T/tmp63eh58pk


In [ ]:
tf.reset_default_graph()

# A single serialized example
#(You can read this from a file using TFRecordReader)
ex= make_example([1, 2, 3], [0, 1, 0]).SerializeToString() 

#Define how to parse the example
context_features= {
    'length': tf.FixedLenFeature([], dtype=tf.int64)
}
sequence_features= {
    'tokens': tf.FixedLenSequenceFeature([], dtype=tf.int64),
    'labels': tf.FixedLenSequenceFeature([], dtype=tf.int64)
}

#Parse the example (returns a dictionary of tensors)
context_parsed, sequence_parsed= tf.parse_single_sequence_example(
    serialized=ex,
    context_features=context_features,
    sequence_features=sequence_features
)
context= tf.contrib.learn.run_n(context_parsed, n=1, )